### Parcialito 2 - Federico del Mazo - 100029

In [ ]:
import networkx as nx
import pandas as pd

df = pd.read_csv('../Shared/World.csv', header=0, names=["source", "target", "_weight"])
Graphtype = nx.Graph()
G = nx.from_pandas_edgelist(df, create_using=Graphtype)

#### Ejercicio 1

- Obtener una visualización de las comunidades presentes en dicha red (indicando el algoritmo utilizado).
- Considerando lo que respondiste en el parcialito 1 (ejercicio 2):
  - Si mencionaste que había homofilia, ¿corresponde por el mismo tipo que mencionaste anteriormente? ¿por
qué?
  - ~~Si mencionaste que no había homofilia (o bien no realizaste el ejercicio), ¿qué tipo de homofilia se puede ver presente?~~
- Obtener los nodos correspondientes a una de las subredes (con al menos 20% de los nodos), y realizar una
visualización de las sub-comunidades presentes.

In [ ]:
from networkx.algorithms import community

louvain = community.louvain_communities(G)
louvain

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import random

plt.figure(figsize=(20,10))

draw_nodes = {}
colors = random.sample(list(mcolors.TABLEAU_COLORS), len(louvain))
for louvaincommunity, color in zip(louvain, colors):
    draw_nodes.update({n: color for n in louvaincommunity})
    
plt.title("Louvain Communities")
nx.draw_networkx(G, 
                 nodelist=draw_nodes.keys(), 
                 node_color=list(draw_nodes.values()), 
                 edgelist=[], 
                 pos=nx.kamada_kawai_layout(G),
                 font_size=10)

In [ ]:
# Quiero comparar mis comunidades contra la homofilia por continentes que encontré en el Parcialito1.
# Hago casi clavado copy-paste de la celda que mapea pais a continente en Parcialito1/parcialito-1.ipynb

df2 = pd.read_csv('../Shared/country_profile_variables.csv', header=0)
region_to_continent = {'SouthernAsia': 'Asia', 'SouthernEurope': 'Europe', 'NorthernAfrica': 'Africa', 'Polynesia': 'Oceania', 'MiddleAfrica': 'Africa', 'Caribbean': 'CentralAmerica', 'SouthAmerica': 'SouthAmerica', 'WesternAsia': 'Asia', 'Oceania': 'Oceania', 'WesternEurope': 'Europe', 'EasternEurope': 'Europe', 'CentralAmerica': 'CentralAmerica', 'WesternAfrica': 'Africa', 'NorthernAmerica': 'NorthernAmerica', 'SouthernAfrica': 'Africa', 'South-easternAsia': 'Asia', 'EasternAfrica': 'Africa', 'NorthernEurope': 'Europe', 'EasternAsia': 'Asia', 'Melanesia': 'Oceania', 'Micronesia': 'Oceania', 'CentralAsia': 'Asia'} 
df2['Continent'] = df2['Region'].map(region_to_continent)
aliases = {"United States of America": "United States", "Viet Nam": "Vietnam", "Bolivia (Plurinational State of)":"Bolivia", "China, Hong Kong SAR": "Hong Kong", "Micronesia (Federated States of)": "Micronesia", "Czechia": "Czech Republic", "Democratic People's Republic of Korea": "South Korea", "Russian Federation": "Russia", "The former Yugoslav Republic of Macedonia": "Macedonia", "Iran (Islamic Republic of)": "Iran", "Venezuela (Bolivarian Republic of)": "Venezuela", "Brunei Darussalam": "Brunei", "Falkland Islands (Malvinas)": "Falkland Islands", "Syrian Arab Republic": "Syria", "Wallis and Futuna Islands": "Wallis and Futuna", "Republic of Korea": "North Korea", "United Republic of Tanzania": "Tanzania"}
df2 = df2[['country', 'Continent']]
df2 = df2.set_index('country').rename(index = aliases)
attributes = df2.to_dict('index')
attributes
nx.set_node_attributes(G, attributes)

manual_attributes = {'Taiwan': 'Asia', 'Netherlands Antilles': 'Europe', "Cote d'Ivoire": 'Africa', 'Congo (Brazzaville)': 'Africa', 'Congo (Kinshasa)': 'Africa', 'Cape Verde': 'Africa', 'Moldova': 'Europe', 'Guernsey': 'Europe', 'Jersey': 'Europe', 'Reunion': 'Europe', 'Virgin Islands': 'CentralAmerica', 'Norfolk Island': 'Oceania', 'Burma': 'Asia', 'Macau': 'Asia', 'Laos': 'Asia', 'East Timor': 'Asia', 'Christmas Island': 'Asia', 'Cocos (Keeling) Islands': 'Asia'}
nx.set_node_attributes(G, {k: {'Continent': v} for k,v in manual_attributes.items()})

In [ ]:
# De cada comunidad de Louvain que tengo... 
#   cuál es mi continente mayoritario? cuantos paises no estan dentro de ese continente?

from collections import Counter

continent_count = Counter(nx.get_node_attributes(G, 'Continent'))
for i, louvaincommunity in enumerate(louvain):
    community_count = {k: v for k,v in continent_count.items() if k in louvaincommunity}
    continents = list(community_count.values())
    majority = max(continents, key=continents.count)
    majority_percent = (len([v for k, v in community_count.items() if v == majority]) / len(louvaincommunity)) * 100
    print(f"La comunidad #{i} tiene {len(louvaincommunity)} paises y consiste de un {majority_percent:.2f}% de paises de {majority}.")

**_Qué podemos decir de todo esto?_**

Claramente los continentes de Europa y Africa se convirtieron en su propia comunidad. Esto muestra que sus paises son bastante similares y cercanos entre sí. De las otras dos comunidades formadas no se puede decir lo mismo con tanta certeza, pero aún así el tener más de la mitad de los paises del mismo continente en ambas comunidades demuestra que aunque sea *algo* de homofilia había.

En conclusión, si, yo creo que las comunidades formadas corresponden al tipo de homofilia por continente, planteada en el parcialito anterior.

In [ ]:
top_community = nx.subgraph(G, max(louvain, key=len))
top_community_louvain = community.louvain_communities(top_community)
plt.figure(figsize=(20,10))

draw_nodes = {}
colors = random.sample(list(mcolors.TABLEAU_COLORS), len(top_community_louvain))
for louvaincommunity, color in zip(top_community_louvain, colors):
    draw_nodes.update({n: color for n in louvaincommunity})
    
plt.title("Louvain Communities (of the Top Louvain Community)")
nx.draw_networkx(top_community, 
                 nodelist=draw_nodes.keys(), 
                 node_color=list(draw_nodes.values()), 
                 edgelist=[], 
                 pos=nx.kamada_kawai_layout(top_community),
                 font_size=10)

#### Ejercicio 2

- Calcular los motifs de hasta 5 nodos de la subred definida en el punto 1.c.
- Calcular el promedio y desvío estandar de los motifs de una red de baseline. Calcular el significant profile de la red, y hacer un gráfico.
- Intentar dar con una explicación del resultado obtenido en el punto anterior

#### Ejercicio 3

Detectar los roles en dicha red utilizando el algoritmo RolX, explicando el resultado obtenido.

In [ ]:
# Ojo, graphrole tiene de dependencia a networkx 2.5, pero los algoritmos copados de comunidades salieron en nx 2.7
# Hay que lograr tener en paralelo nx > 2.7 y graphrole:
# - Asegurarse de estar en py3.9, en vez de py3.10
# - Clonar el repo de graphrole
# - Modificar el requirements.txt del repo clonado para ponerle nx 2.8
# - Hacer `pip install ./GraphRole`

from graphrole import RecursiveFeatureExtractor, RoleExtractor

feature_extractor = RecursiveFeatureExtractor(G)
features = feature_extractor.extract_features()
role_extractor = RoleExtractor(n_roles=None)
role_extractor.extract_role_factors(features)

In [ ]:
roles = {}

for country, role in role_extractor.roles.items():
    roles.setdefault(role, set()).add(country)

roles

In [ ]:
features.sample(3)

In [ ]:
plt.figure(figsize=(20,10))

draw_nodes = {}
colors = random.sample(list(mcolors.TABLEAU_COLORS), len(roles))
for role, color in zip(roles, colors):
    draw_nodes.update({n: color for n in roles[role]})
    
plt.title("Graph Roles")
nx.draw_networkx(G, 
                 nodelist=draw_nodes.keys(), 
                 node_color=list(draw_nodes.values()), 
                 edgelist=[], 
                 with_labels=False)

Como podemos ver, `graphrole` (aplicando `RolX`) nos agrupó los nodos de nuestro grafo en distintos roles, basandose en features locales y regionales. Es decir, nos encontró nodos con posiciones similares, pero no necesariamente cercanos entre sí.

Se puede ver que uno de los roles encontrados claramente se refiere a los periféricos de nuestra red, mientras que los otros se refieren a nodos más centrales, como si fuese una cebolla 🧅.